In [1]:
import numpy as np
import pandas as pd

In [30]:
fao_stat = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Production_Crops_Livestock_E_All_Data_analyze.csv")
crop_class = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/Crop Classification_latest.xlsx", engine="openpyxl", skiprows=1)
crop_class = crop_class.drop('Unnamed: 0', axis=1)
crop_class.rename(columns={'FAO Crops': 'Item'}, inplace=True)

In [31]:
fao_stat1 = fao_stat[(fao_stat['Element Code'] == 5510) & (fao_stat['Item'] != 'Oil palm fruit')] #Palm oil
# fao_stat2 = fao_stat[(fao_stat['Element Code'] == 5510) & (fao_stat['Item'] != 'Palm oil')]

In [32]:
fao_stat1['Item'] = fao_stat1['Item'].replace('Palm oil', 'Oil palm fruit')

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_13700/2632990359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fao_stat1['Item'] = fao_stat1['Item'].replace('Palm oil', 'Oil palm fruit')


In [36]:
fao_stat1 = fao_stat1[~fao_stat1["Area"].isin(["Americas", "Asia", "Australia and New Zealand", "Africa", "Belgium-Luxembourg", "Central America", "Central Asia", "Caribbean", "Czechoslovakia", "Eastern Africa", "Eastern Asia",
 "Eastern Europe", "European Union (27)","Land Locked Developing Countries", "Least Developed Countries", "Europe", "Low Income Food Deficit Countries", "Melanesia", "Middle Africa", 
  "Net Food Importing Developing Countries", "Northern Africa", "Northern America", "Northern Europe", "Oceania", "Polynesia", "Small Island Developing States", "Serbia and Montenegro", "China",
  "South-eastern Asia", "Southern Africa", "Southern Asia", "Southern Europe", "Sudan (former)", "South America", "Western Africa", "Western Asia", "Western Europe", "World", "Yugoslav SFR"])]

In [53]:
fao_stat_production = fao_stat1[fao_stat1['Element'] == "Production"]
fao_stat_production = fao_stat_production.fillna(0)
fao_stat_production = fao_stat_production.merge(crop_class, on='Item')
fao_stat_production.head()

,Area Code,Area Code (M49),Area,Item Code,Item Code (CPC),Item,Element Code,Element,Unit,Year,...,photosynthesis pathway,Alternative Class (C3 nitrogen fixing/ legumes crops) & (CAM to C4),Monfreda annual/ perennial,LUH2Class,IMAGE Classification,Explicitly in IMAGE class,Hong 2021 Classification,Monfreda 2008\n Group,Monfreda Crop Name,Herbaceous/Shrub/Tree
0,2,'004,Afghanistan,221,'01371,"Almonds, in shell",5510,Production,t,1961,...,C3,NaN,perennial,c3per,Vegetables & fruits,YES,Pulses,tree nuts,Almonds,tree
1,2,'004,Afghanistan,711,'01654,"Anise, badian, coriander, cumin, caraway, fenn...",5510,Production,t,1961,...,C3,NaN,annual,c3ann,"Other non-food, luxury, spices",YES,Spices,other,"Anise, badian, and fennel",herbaceous
2,2,'004,Afghanistan,515,'01341,Apples,5510,Production,t,1961,...,C3,NaN,perennial,c3per,Vegetables & fruits,YES,Fruit,fruit,Apples,tree
3,2,'004,Afghanistan,526,'01343,Apricots,5510,Production,t,1961,...,C3,NaN,perennial,c3per,Vegetables & fruits,YES,Fruit,fruit,Apricots,tree
4,2,'004,Afghanistan,44,'0115,Barley,5510,Production,t,1961,...,C3,NaN,annual,c3ann,Other temperate cereals,YES,Cereals,cereals,Barley,herbaceous


In [54]:
fao_stat_prod = fao_stat_production.copy()

In [55]:
fao_stat_prod1 = fao_stat_prod.drop_duplicates(subset=['Area', 'Item', 'Year', 'Element', 'IMAGE Classification'])

In [56]:
unique_values = fao_stat_prod1['Item'].unique()

In [57]:
fao_stat_prod1['IMAGE Classification'] = fao_stat_prod1['IMAGE Classification'].apply(lambda x: x.replace('and', '&').replace('Plant-based fibers', 'Plant based fibres').replace('Oil, palm fruit', 'Oil & palm fruit').replace('Other non-food, luxury, spices', 'Other non-food & luxury & spices').replace('Vegetables or fruits', 'Vegetables & fruits'))

In [58]:
fao_stat_prod1_columns = fao_stat_prod1[['Area', 'Item', 'Year','IMAGE Classification', 'Value']]
# fao_stat_prod1_columns = fao_stat_prod1_columns.rename(columns={"IMAGE Classification": "IMAGE Classification"})
fao_stat_prod1_columns = fao_stat_prod1_columns[fao_stat_prod1_columns['Year'] >= 1968]
# fao_stat_prod1_columns.to_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/fao_stat_prod1_columns.csv", index=False)

In [63]:
mask = (fao_stat_prod1_columns['Area'] == "Russian Federation") & (fao_stat_prod1_columns['Year'] <= 1991)
fao_stat_prod1_columns.loc[mask, ['Item', 'Year', 'IMAGE Classification', 'Value']] = fao_stat_prod1_columns[(fao_stat_prod1_columns['Area'] == "USSR") & (fao_stat_prod1_columns['Year'] <= 1991)][['Item', 'Year', 'IMAGE Classification', 'Value']]

In [64]:
mask = (fao_stat_prod1_columns['Area'] == "Russian Federation") & (fao_stat_prod1_columns['Year'] <= 1991)
ussr_mask = (fao_stat_prod1_columns['Area'] == "USSR") & (fao_stat_prod1_columns['Year'] <= 1991)

ussr_values = fao_stat_prod1_columns.loc[ussr_mask, ['Item', 'Year', 'IMAGE Classification', 'Value']].values
fao_stat_prod1_columns.loc[mask, ['Item', 'Year', 'IMAGE Classification', 'Value']] = ussr_values[:len(mask[mask])]

In [65]:
fao_stat_prod1_columns[(fao_stat_prod1_columns['Area'] == 'USSR' ) & (fao_stat_prod1_columns['Year'] <= 1991)]

,Area,Item,Year,IMAGE Classification,Value
85126,USSR,"Almonds, in shell",1968.0,Vegetables & fruits,3700.0
85127,USSR,Apples,1968.0,Vegetables & fruits,3816000.0
85128,USSR,Apricots,1968.0,Vegetables & fruits,160000.0
85129,USSR,Barley,1968.0,Other temperate cereals,26592000.0
85130,USSR,"Beans, dry",1968.0,Pulses,58000.0
...,...,...,...,...,...
331297,USSR,Unmanufactured tobacco,1991.0,Other non-food & luxury & spices,217496.0
331298,USSR,Vetches,1991.0,Pulses,630500.0
331299,USSR,"Walnuts, in shell",1991.0,Vegetables & fruits,96000.0
331300,USSR,Watermelons,1991.0,Vegetables & fruits,4950000.0


In [66]:
#menghapus USSR (posisi df sebelum dihapus ada double USSR dan Russia Federation yg diisi dg nilai USSR)
fao_stat_prod1_columns_update = fao_stat_prod1_columns[~fao_stat_prod1_columns["Area"].isin(["USSR"])]

In [67]:
#validasi -> sdh tidak ada lagi data USSR
fao_stat_prod1_columns_update[(fao_stat_prod1_columns_update['Area'] == 'USSR' ) & (fao_stat_prod1_columns_update['Year'] <= 1991)]

,Area,Item,Year,IMAGE Classification,Value


In [68]:
fao_stat_prod1_columns_drop = fao_stat_prod1_columns_update.drop_duplicates(subset=['Area', 'Item', 'Year', 'IMAGE Classification', 'Value'])
fao_stat_prod1_columns_drop

,Area,Item,Year,IMAGE Classification,Value
74900,Afghanistan,"Almonds, in shell",1968.0,Vegetables & fruits,0.00
74901,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",1968.0,Other non-food & luxury & spices,0.00
74902,Afghanistan,Apples,1968.0,Vegetables & fruits,27900.00
74903,Afghanistan,Apricots,1968.0,Vegetables & fruits,59400.00
74904,Afghanistan,Barley,1968.0,Other temperate cereals,361000.00
...,...,...,...,...,...
663395,Micronesia,Rice,2022.0,Rice,178.30
663396,Micronesia,Sorghum,2022.0,Tropical cereals,16.96
663397,Micronesia,Sweet potatoes,2022.0,Tropical roots & tubers,3118.06
663398,Micronesia,Taro,2022.0,Tropical roots & tubers,1959.75


In [69]:
step1_copy = fao_stat_prod1_columns_drop.copy()
step1_copy['Value'] = step1_copy['Value'].fillna(0)
step1_copy['Value'] = 0
step1_copy['Value'] = fao_stat_prod1_columns_drop.groupby(['Area', "Year", 'IMAGE Classification'])['Value'].transform('sum')
step1_copy

,Area,Item,Year,IMAGE Classification,Value
74900,Afghanistan,"Almonds, in shell",1968.0,Vegetables & fruits,1336800.00
74901,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",1968.0,Other non-food & luxury & spices,0.00
74902,Afghanistan,Apples,1968.0,Vegetables & fruits,1336800.00
74903,Afghanistan,Apricots,1968.0,Vegetables & fruits,1336800.00
74904,Afghanistan,Barley,1968.0,Other temperate cereals,361000.00
...,...,...,...,...,...
663395,Micronesia,Rice,2022.0,Rice,178.30
663396,Micronesia,Sorghum,2022.0,Tropical cereals,16.96
663397,Micronesia,Sweet potatoes,2022.0,Tropical roots & tubers,14114.66
663398,Micronesia,Taro,2022.0,Tropical roots & tubers,14114.66


In [70]:
step1_copy_drop = step1_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])
step1_copy_drop

,Area,Item,Year,IMAGE Classification,Value
74900,Afghanistan,"Almonds, in shell",1968.0,Vegetables & fruits,1336800.00
74901,Afghanistan,"Anise, badian, coriander, cumin, caraway, fenn...",1968.0,Other non-food & luxury & spices,0.00
74904,Afghanistan,Barley,1968.0,Other temperate cereals,361000.00
74908,Afghanistan,Linseed,1968.0,Temperate oil crops,36100.00
74909,Afghanistan,Maize (corn),1968.0,Maize,773000.00
...,...,...,...,...,...
663380,Micronesia,"Coconuts, in shell",2022.0,Tropical oil crops,301379.02
663382,Micronesia,Edible roots and tubers with high starch or in...,2022.0,Temperate roots & tubers,0.00
663386,Micronesia,Maize (corn),2022.0,Maize,88.99
663395,Micronesia,Rice,2022.0,Rice,178.30


In [71]:
#drop kolom item (tdk diperlukan)
step1_copy_columns = step1_copy_drop[['Area', 'Year', 'IMAGE Classification', 'Value']]

In [72]:
step2_copy = step1_copy_columns.copy()
eight_year_ranges = [(y, y+4) for y in range(1968, 2022, 5)]

for year in eight_year_ranges:
    mask = (step1_copy_columns['Year'] >= year[0]) & (step1_copy_columns['Year'] <= year[1]) #5years
    step2_copy.loc[mask, 'Value'] = step1_copy_columns[mask].groupby(['Area', 'IMAGE Classification'])['Value'].transform('mean')

step2_copy

,Area,Year,IMAGE Classification,Value
74900,Afghanistan,1968.0,Vegetables & fruits,1298080.000
74901,Afghanistan,1968.0,Other non-food & luxury & spices,0.000
74904,Afghanistan,1968.0,Other temperate cereals,360200.000
74908,Afghanistan,1968.0,Temperate oil crops,34100.000
74909,Afghanistan,1968.0,Maize,723000.000
...,...,...,...,...
663380,Micronesia,2022.0,Tropical oil crops,300290.804
663382,Micronesia,2022.0,Temperate roots & tubers,0.000
663386,Micronesia,2022.0,Maize,89.180
663395,Micronesia,2022.0,Rice,178.818


In [73]:
#data cleaning hapus nilai average sama yg di write tiap tahun
step2_copy_drop = step2_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])
step2_copy_drop

,Area,Year,IMAGE Classification,Value
74900,Afghanistan,1968.0,Vegetables & fruits,1298080.000
74901,Afghanistan,1968.0,Other non-food & luxury & spices,0.000
74904,Afghanistan,1968.0,Other temperate cereals,360200.000
74908,Afghanistan,1968.0,Temperate oil crops,34100.000
74909,Afghanistan,1968.0,Maize,723000.000
...,...,...,...,...
663380,Micronesia,2022.0,Tropical oil crops,300290.804
663382,Micronesia,2022.0,Temperate roots & tubers,0.000
663386,Micronesia,2022.0,Maize,89.180
663395,Micronesia,2022.0,Rice,178.818


In [74]:
step3_copy = step2_copy_drop.copy()

eight_year_ranges = [(y, y+4) for y in range(1968, 2022, 5)]
five_yearIncrements = range(1970, 2025, 5)
for year, five_year in zip(eight_year_ranges, five_yearIncrements):
    year_idx = step2_copy_drop[(step2_copy_drop['Year'] >= year[0]) & (step2_copy_drop['Year'] <= year[1])]['Year'].index
    step3_copy.loc[year_idx, 'Year'] = five_year
step3_copy

,Area,Year,IMAGE Classification,Value
74900,Afghanistan,1970.0,Vegetables & fruits,1298080.000
74901,Afghanistan,1970.0,Other non-food & luxury & spices,0.000
74904,Afghanistan,1970.0,Other temperate cereals,360200.000
74908,Afghanistan,1970.0,Temperate oil crops,34100.000
74909,Afghanistan,1970.0,Maize,723000.000
...,...,...,...,...
663380,Micronesia,2020.0,Tropical oil crops,300290.804
663382,Micronesia,2020.0,Temperate roots & tubers,0.000
663386,Micronesia,2020.0,Maize,89.180
663395,Micronesia,2020.0,Rice,178.818


In [75]:
step3_copy = step3_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])
step3_copy.head()

,Area,Year,IMAGE Classification,Value
74900,Afghanistan,1970.0,Vegetables & fruits,1298080.0
74901,Afghanistan,1970.0,Other non-food & luxury & spices,0.0
74904,Afghanistan,1970.0,Other temperate cereals,360200.0
74908,Afghanistan,1970.0,Temperate oil crops,34100.0
74909,Afghanistan,1970.0,Maize,723000.0


In [76]:
#drop yg duplikat
step3_copy_drop = step3_copy.drop_duplicates(subset=['Area', 'Year', 'IMAGE Classification', 'Value'])
step3_copy_drop

,Area,Year,IMAGE Classification,Value
74900,Afghanistan,1970.0,Vegetables & fruits,1298080.000
74901,Afghanistan,1970.0,Other non-food & luxury & spices,0.000
74904,Afghanistan,1970.0,Other temperate cereals,360200.000
74908,Afghanistan,1970.0,Temperate oil crops,34100.000
74909,Afghanistan,1970.0,Maize,723000.000
...,...,...,...,...
620580,Micronesia,2020.0,Tropical oil crops,300290.804
620582,Micronesia,2020.0,Temperate roots & tubers,0.000
620586,Micronesia,2020.0,Maize,89.180
620595,Micronesia,2020.0,Rice,178.818


In [77]:
fao_stat_copy = step3_copy_drop.copy()

prubahan_nama = {
    "China, Hong Kong SAR": "Hong Kong",
    "China, Macao SAR": "Macao", 
    "China, Taiwan Province of": "Taiwan,  Province of China",
    "China, mainland": "China",
    "Democratic People's Republic of Korea": "Korea (Democratic People's Republic of)",
    "Democratic Republic of the Congo": "Congo, Democratic Republic of the",
    "Micronesia": "Micronesia (Federated States of)",
    "Micronesia (Federated States of) (Federated States of)": "Micronesia (Federated States of)",
    "Netherlands (Kingdom of the)": "Netherlands",
    "Republic of Korea": "Korea, Republic of",
    "Republic of Moldova": "Moldova, Republic of",
    "Palestine" : "Palestine, State of",
    "Türkiye": "Turkey",
    "United Republic of Tanzania": "Tanzania, United Republic of",
    "Ethiopia PDR" : "Eritrea"
}
keys_list = list(prubahan_nama.keys())
values_list = list(prubahan_nama.values())

# substitute
for i in range(len(keys_list)):
    fao_stat_copy['Area'] = fao_stat_copy['Area'].astype(str).str.replace(keys_list[i], values_list[i])

In [78]:
fao_stat_copy.to_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/FAO_Production_Crops_minyak.csv", index=False)